# Backtesting assets and a benchmark both b&h and RSI strategy

### In this test we are using vectorbt and pandas_ta (pandas_ta is optional, but is useful

In [ ]:
# Define functions to connect to Google and change directories
def connectDrive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

def changeDirectory(path):
    import os
    original_path = os.getcwd()
    os.chdir(path)
    new_path = os.getcwd()
    print("Original path: ",original_path)
    print("New path: ",new_path)

# Connect to Google Drive
connectDrive()

# Change path
changeDirectory("/content/drive/My Drive/github/find_p/")

In [ ]:
!pip install vectorbt
!pip install pandas_ta
!pip install yfinance
!pip install plotly
!pip install numba
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca

In [1]:
import asyncio
import itertools
from datetime import datetime

from IPython import display

import numpy as np
import pandas as pd
import vectorbt as vbt

from pdta_vt_utils import *
import warnings

import plotly.graph_objects as go
%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)

# Configuration of the backtesting platform

### Important step that need check before any experiment

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1000
vbt.settings.portfolio["fees"] = 0.0025 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025 # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings


Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,1000
size,inf
size_type,amount
fees,0.0025
fixed_fees,0.0
slippage,0.0025
reject_prob,0.0
min_size,0.0


# Setting of the assets to test

### It is not dynamic and do not offer any advantage but is a way to have the data collected

In [3]:
benchmark_tickers = ["SPY", "QQQ"]
asset_tickers = ["AAPL", "TSLA", "TWTR", "SPXL", "^GSPC", "SPY"]
all_tickers = benchmark_tickers + asset_tickers

print("Tickers by index #")
print("="*100)
print(f"Benchmarks: {', '.join([f'{k}: {v}' for k,v in enumerate(benchmark_tickers)])}")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")
print(f"       All: {', '.join([f'{k}: {v}' for k,v in enumerate(all_tickers)])}")
print("="*100)
benchmarks = dl(benchmark_tickers, lc_cols=True)
assets = dl(asset_tickers, lc_cols=True)

Tickers by index #
Benchmarks: 0: SPY, 1: QQQ
    Assets: 0: AAPL, 1: TSLA, 2: TWTR, 3: SPXL
       All: 0: SPY, 1: QQQ, 2: AAPL, 3: TSLA, 4: TWTR, 5: SPXL


In [4]:
########### You need to set this #############################
# Numero del indice que se quiere ver tanto para exploracion com para benchmark e.j. spy i=5
benchmark_name = benchmark_tickers[0] # Change index for different benchmark
asset_name = asset_tickers[3] # Change index for different symbol
#################################################################

print("="*100)
print(f"Selected Benchmark | Asset: {benchmark_name} | {asset_name}")
start_date = datetime(2010, 1, 1) # Adjust as needed
start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
end_date = datetime(2015, 1, 1)   # Adjust as needed
end_date = pd.to_datetime(end_date).tz_localize('America/New_York')
print("Available Data:")
print("="*100)
print(f"Benchmarks: {', '.join(benchmarks.keys())}")
print(f"Assets: {', '.join(assets.keys())}")
print("="*100)

benchmarkdf = benchmarks[benchmark_name]
assetdf     = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
crs = ''
if common_range:
    crs = f" from {start_date} to {end_date}"
    benchmarkdf = dtmask(benchmarkdf, start_date, end_date)
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
benchmarkdf.name = benchmark_name
assetdf.name = asset_name
print(f"Analysis of: {benchmarkdf.name} and {assetdf.name}{crs}")

Selected Benchmark | Asset: SPY | SPXL
[i] Downloading: SPY, QQQ
[+] SPY(7565, 8) Monday February 13, 2023, NYSE: 10:21:16
[+] QQQ(6023, 8) Monday February 13, 2023, NYSE: 10:21:16
[*] Download Complete

[i] Downloading: AAPL, TSLA, TWTR, SPXL
[+] AAPL(10632, 7) Monday February 13, 2023, NYSE: 10:21:17
[+] TSLA(3179, 7) Monday February 13, 2023, NYSE: 10:21:17
[+] TWTR(2259, 7) Monday February 13, 2023, NYSE: 10:21:18
[+] SPXL(3592, 8) Monday February 13, 2023, NYSE: 10:21:18
[*] Download Complete



In [7]:
benchmarkdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2015-01-02 00:00:00-05:00,178.284274,178.716206,176.383762,177.463593,121465900,0.0,0.0,0.0
2015-01-05 00:00:00-05:00,176.375178,176.547948,173.939087,174.258713,169632600,0.0,0.0,0.0
2015-01-06 00:00:00-05:00,174.578302,175.122541,171.788025,172.617340,209151400,0.0,0.0,0.0
2015-01-07 00:00:00-05:00,173.999533,175.122559,173.533052,174.768372,125346700,0.0,0.0,0.0
2015-01-08 00:00:00-05:00,176.236926,178.094242,176.219658,177.869629,147217800,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2020-12-24 00:00:00-05:00,357.439225,358.361773,356.827462,358.332642,26457900,0.0,0.0,0.0
2020-12-28 00:00:00-05:00,360.993450,361.818883,360.342835,361.411041,39000400,0.0,0.0,0.0
2020-12-29 00:00:00-05:00,363.003597,363.188107,360.109735,360.721527,53680500,0.0,0.0,0.0


In [8]:
assetdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2015-01-02 00:00:00-05:00,20.830924,20.985625,20.172265,20.542614,4748800,0.0,0.0,0.0
2015-01-05 00:00:00-05:00,20.165229,20.233204,19.328429,19.440941,7138400,0.0,0.0,0.0
2015-01-06 00:00:00-05:00,19.558139,19.724561,18.611172,18.904169,9798400,0.0,0.0,0.0
2015-01-07 00:00:00-05:00,19.356557,19.701122,19.183102,19.607363,7011600,0.0,0.0,0.0
2015-01-08 00:00:00-05:00,20.097252,20.711375,20.078502,20.650431,7421200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2020-12-24 00:00:00-05:00,68.667941,69.185585,68.339432,69.185585,2141500,0.0,0.0,0.0
2020-12-28 00:00:00-05:00,70.728581,71.176542,70.340346,70.937630,3672000,0.0,0.0,0.0
2020-12-29 00:00:00-05:00,71.883326,71.992829,70.191020,70.559341,4877600,0.0,0.0,0.0


In [9]:
def stochastic(df_dict, lookback=14, k=3, d=3):
    """function to calculate Stochastic Oscillator
    lookback = lookback period
    k and d = moving average window for %K and %D"""
    df_dict["HH"] = df_dict["high"].rolling(lookback).max()
    df_dict["LL"] = df_dict["low"].rolling(lookback).min()
    df_dict["%K"] = (100 * (df_dict["close"] - df_dict["LL"])/(df_dict["HH"]-df_dict["LL"])).rolling(k).mean()
    df_dict["%D"] = df_dict["%K"].rolling(d).mean()
    df_dict.drop(["HH","LL"], axis=1, inplace=True)

def stochastic_oscilator_momentum(dfi, win_h=14, win_l=14):
    high14 = dfi['High'].rolling(win_h).max()
    low14 = dfi['Low'].rolling(win_l).min()
    dfi['perc_k'] = (dfi['Close'] - low14) * 100 / (high14 - low14)
    dfi['perc_d'] = dfi['perc_k'].rolling(3).mean()
    return dfi

def trends(df: pd.DataFrame, wind_low: int = 8, wind_high:int = 5):
    stochastic_oscilator_momentum(df, wind_high, wind_low)
    df['TS_Entries'] = False
    df['TS_Exits'] = False
    df['TS_Trend'] = False
    # k crossing d d>k time to sell (over bought), d<k time to buy (over sold)
    for i in range(0, len(df)):
        if df.perc_d[i] > 80 < df.perc_k.iloc[i] < df.perc_d.iloc[i]:
            df.TS_Exits.iloc[i] = True
            df.TS_Trend.iloc[i] = True
        if df.perc_k.iloc[i] < 20 > df.perc_d.iloc[i]  < df.perc_k.iloc[i]:
            df.TS_Entries.iloc[i] = True
            df.TS_Trend.iloc[i] = True
    return df

In [10]:
####### Configuration for the strategy ###########
# You can chenge this, remember for example you can set the default values
# We are adding stop loss as a parameter here
###################################################
k_window = 8
d_window = 3
d_ewm = 3
sl_stop = 0.002
below = 20
above = 80
#######################################################


benchmark_ind = vbt.STOCH.run(benchmarkdf.High, benchmarkdf.Low, benchmarkdf.Close, k_window=k_window, d_window=d_window, d_ewm=d_ewm)

benchmark_entries_st = (benchmark_ind.percent_k_below(below) & benchmark_ind.percent_d_below(below)) & benchmark_ind.percent_d_crossed_below(benchmark_ind.percent_k)
benchmark_exits_st =   (benchmark_ind.percent_k_above(above) & benchmark_ind.percent_d_above(above)) & benchmark_ind.percent_d_crossed_above(benchmark_ind.percent_k)



asset_ind = vbt.STOCH.run(assetdf.High, assetdf.Low, assetdf.Close, k_window=k_window, d_window=d_window, d_ewm=d_ewm)

asset_entries_st = (asset_ind.percent_k_below(below) & asset_ind.percent_d_below(below)) & asset_ind.percent_d_crossed_below(asset_ind.percent_k)
asset_exits_st =   (asset_ind.percent_k_above(above) & asset_ind.percent_d_above(above)) & asset_ind.percent_d_crossed_above(asset_ind.percent_k)


In [11]:
# # trade_offset = 0 for Live Signals (close is last price)
# # trade_offset = 1 for Backtesting
LIVE = 0


# Backtest the strategy by signals

### Signal Portfolios with their Last 'k' Trades and Performance Statistics

In [12]:
# Benchmark Portfolio from Trade Signals
benchmarkpf_signals = vbt.Portfolio.from_signals(benchmarkdf.Close,
                                                 entries=benchmark_entries_st,
                                                 exits=benchmark_exits_st,
                                                 sl_stop=sl_stop)
trade_table(benchmarkpf_signals, k=5)
combine_stats(benchmarkpf_signals, benchmarkdf.name, "Long Strategy", LIVE)


Last 5 of 4 Trades
   status  direction      size  entry_price  exit_price    return        pnl  \
0       1          0  5.501102   181.328442  179.114090 -0.017181 -17.138453   
1       1          0  5.773744   169.804985  166.489578 -0.024476 -23.996504   
2       1          0  4.321353   221.336676  227.539939  0.022956  21.957103   
3       1          0  3.782526   258.656840  270.798815  0.041825  40.920636   

   entry_fees  exit_fees  
0    2.493766   2.463312  
1    2.451026   2.403171  
2    2.391185   2.458201  
3    2.445941   2.560759  



Run Time                      Monday February 13, 2023, NYSE: 10:21:36
Mode                                                              TEST
Strategy                                                 Long Strategy
Direction                                                     longonly
Symbol                                                             SPY
Fees [%]                                                          0.25
Slippage [%]                                                      0.25
Accumulate                                                       False
Start                                        2015-01-02 00:00:00-05:00
End                                          2020-12-31 00:00:00-05:00
Period                                              1511 days 00:00:00
Start Value                                                     1000.0
End Value                                                  1021.742781
Total Return [%]                                              2.174278
Benchm

In [13]:
# Asset Portfolio from Trade Signals
assetpf_signals = vbt.Portfolio.from_signals(assetdf.Close,
                                             entries=asset_entries_st,
                                             exits=asset_exits_st,
                                             sl_stop=sl_stop)
trade_table(assetpf_signals, k=5)
combine_stats(assetpf_signals, assetdf.name, "Long Strategy", LIVE)


Last 5 of 8 Trades
   status  direction       size  entry_price  exit_price    return  \
3       1          0  26.818371    32.267953   35.202678  0.085721   
4       1          0  30.092177    31.216413   29.214384 -0.068974   
5       1          0  21.079371    41.497432   47.940099  0.149866   
6       1          0  20.101904    50.020484   49.339806 -0.018574   
7       1          0  37.861886    26.065191   22.078815 -0.157556   

          pnl  entry_fees  exit_fees  
3   74.180893    2.163435   2.360196  
4  -64.791656    2.348425   2.197811  
5  131.094132    2.186849   2.526368  
6  -18.676253    2.513767   2.479560  
7 -155.488761    2.467193   2.089864  



Run Time                      Monday February 13, 2023, NYSE: 10:21:40
Mode                                                              TEST
Strategy                                                 Long Strategy
Direction                                                     longonly
Symbol                                                            SPXL
Fees [%]                                                          0.25
Slippage [%]                                                      0.25
Accumulate                                                       False
Start                                        2015-01-02 00:00:00-05:00
End                                          2020-12-31 00:00:00-05:00
Period                                              1511 days 00:00:00
Start Value                                                     1000.0
End Value                                                   833.855726
Total Return [%]                                            -16.614427
Benchm

# Signal Plots

### Benchmark

In [15]:
assetpf_signals.plot().show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '44697ccd-dc52-40f8-8f28-aff668e6516a',
              'x': array([datetime.datetime(2015, 1, 2, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2015, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2015, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2020, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2020, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2020, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([20.54261398, 19.44094086, 18.90416908, ..., 70.55934143, 70.79826355,
                          71.92314148]),
              'yaxis': 'y'},
             {'customdata': array([[ 0.        , 47.07777505,  2.49376559],
                                   [ 2.        , 59.26972867,  2.39690853],
                                   [ 4.        , 59.76821753,  2.26047106],
                                   [ 6.        , 26.81837068,  2.16343484],
                                   [ 8.        , 30.09217708,  2.3484246 ],
                                   [10.        , 21.07937061,  2.1868494 ],
                                   [12.        , 20.10190423,  2.51376743],
                                   [14.        , 37.86188583,  2.46719323]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'legendgroup': '1',
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy',
              'type': 'scatter',
              'uid': '425dc9ee-2709-4831-b7dc-56b1db7d3e1e',
              'x': array([datetime.datetime(2015, 6, 9, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2016, 1, 11, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2016, 1, 19, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2017, 8, 21, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2018, 12, 20, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2019, 6, 3, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2020, 2, 28, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2020, 3, 17, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([21.18847446, 16.17627467, 15.12824812, 32.26795345, 31.21641339,
                          41.49743247, 50.02048368, 26.06519121]),
              'yaxis': 'y'},
             {'customdata': array([[ 1.        , 47.07777505,  2.4089231 ],
                                   [ 3.        , 59.26972867,  2.27180174],
                                   [ 5.        , 59.76821753,  2.17427913],
                                   [ 7.        , 26.81837068,  2.36019615],
                                   

# Just Continue runing if you are satisfied with the backtest, if not run first the optimization and find the good parameters for the indicator.

In [ ]:
benchmarkpf_signals.trades.plot(title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.value().vbt.plot(title=f"{benchmarkdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.drawdown().vbt.plot(title=f"{benchmarkdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.trades.plot_pnl(title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.returns().vbt.plot(title=f"{benchmarkdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.cash().vbt.plot(title=f"{benchmarkdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
total_assetfees = benchmarkpf_signals.trades.records_readable["Entry Fees"] + benchmarkpf_signals.trades.records_readable["Exit Fees"]
total_assetfees.vbt.plot(title=f"{benchmarkdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

### Asset

In [ ]:
assetpf_signals.trades.plot(title=f"{assetdf.name} | Trades", height=cheight, width=cwidth).show_png()

In [ ]:
assetpf_signals.value().vbt.plot(title=f"{assetdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.drawdown().vbt.plot(title=f"{assetdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.trades.plot_pnl(title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.returns().vbt.plot(title=f"{assetdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.cash().vbt.plot(title=f"{assetdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
total_assetfees = assetpf_signals.trades.records_readable["Entry Fees"] + assetpf_signals.trades.records_readable["Exit Fees"]
total_assetfees.vbt.plot(title=f"{assetdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

# Uncomment and run the follwing just if you want a benchmark with buy and hold in details, remember the first plot containg the buy and hold of the Asset

In [ ]:
# Benchmark Buy and Hold (BnH) Strategy
# benchmarkpf_bnh = vbt.Portfolio.from_holding(benchmarkdf.Close)
# print(trade_table(benchmarkpf_bnh))
# combine_stats(benchmarkpf_bnh, benchmarkdf.name, "Buy and Hold", LIVE)
#### Benchmark Buy and Hold Plots #######################
# vbt.settings.set_theme("seaborn")
# benchmarkpf_bnh.trades.plot(title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth).show_png()
# benchmarkpf_bnh.value().vbt.plot(title=f"{benchmarkdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.drawdown().vbt.plot(title=f"{benchmarkdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.trades.plot_pnl(title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.returns().vbt.plot(title=f"{benchmarkdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.cash().vbt.plot(title=f"{benchmarkdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# total_assetfees = benchmarkpf_bnh.trades.records_readable["Entry Fees"] + benchmarkpf_bnh.trades.records_readable["Exit Fees"]
# total_assetfees.vbt.plot(title=f"{benchmarkdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
# Asset Buy and Hold (BnH) Strategy
# assetpf_bnh = vbt.Portfolio.from_holding(assetdf.Close)
# print(trade_table(assetpf_bnh))
# combine_stats(assetpf_bnh, assetdf.name, "Buy and Hold", LIVE)
### Asset Buy and Hold ############################
# vbt.settings.set_theme("seaborn")
# assetpf_bnh.trades.plot(title=f"{assetdf.name} | Trades", height=cheight, width=cwidth).show_png()
# assetpf_bnh.value().vbt.plot(title=f"{assetdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.drawdown().vbt.plot(title=f"{assetdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.trades.plot_pnl(title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.returns().vbt.plot(title=f"{assetdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.cash().vbt.plot(title=f"{assetdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# total_assetfees = assetpf_bnh.trades.records_readable["Entry Fees"] + assetpf_bnh.trades.records_readable["Exit Fees"]
# total_assetfees.vbt.plot(title=f"{assetdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()